## PHASE I

In [ ]:
pip install hazm

In [29]:
from __future__ import unicode_literals
import pandas as pd
import numpy as np
from hazm import *
import collections 
import string
import operator
import datetime
import json
import math
import copy
import re

In [ ]:
# upload the file 
from google.colab import files

uploaded = files.upload()


In [3]:
# Read file from the Google colud
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1jcbbbPTNnQ3NKrPJJL_oc9vrFvHH4UuR'
downloaded = drive.CreateFile({'id': file_id})

In [4]:
downloaded.GetContentFile('/IR1_7k_news.xlsx')

In [35]:
# read data
# df = pd.read_excel("./IR1_7k_news.xlsx")
df = pd.read_excel("/IR1_7k_news.xlsx")
urls = df['url']
content = df['content']
title = df['title']
# normalize data by use hazm
normalizer = Normalizer()
print(content[11])
for i in range(len(content)):
  content[i] = normalizer.normalize(content[i])
copy_content = content
print(content[11])

به گزارش خبرگزاری فارس از **اصفهان،** دیدار تیم‌های ذوب‌آهن و استقلال با 2 گل
به سود شاگردان [فرهاد مجیدی](https://www.farsnews.ir/special/فرهاد مجیدی) به
پایان رسید.

بعد از این مسابقه [رشید مظاهری](https://search.farsnews.ir/?q=رشید
مظاهری&o=on) در رختکن ذوب‌آهن حاضر شد و با بازیکنان تیم میزبان خوش وبش کرد.
مظاهری بعد از خروج از رختکن به درخواست خبرنگاران برای مصاحبه پاسخ منفی داد و
گفت آنقدر مشکل روحی و روانی دارم که اصلاً نمی‌توانم مصاحبه کنم.

به گزارش فارس، ظاهراً طی روزهای گذشته مشکلاتی برای دروازه‌بان استقلال در
تمرینات این تیم به وجود آمده که وی [به صورت](https://search.farsnews.ir/?q=به
صورت&o=on) سربسته مقابل خبرنگاران به این موضوع اشاره کرده است.

انتهای پیام/


به گزارش خبرگزاری فارس از **اصفهان، ** دیدار تیم‌های ذوب‌آهن و استقلال با ۲ گل
به سود شاگردان [فرهاد مجیدی] (https: //www. farsnews. ir/special/فرهاد مجیدی) به
پایان رسید. 

بعد از این مسابقه [رشید مظاهری] (https: //search. farsnews. ir/?q=رشید
مظاهری&o=on) در رختکن ذوب‌آهن حاضر شد و با بازیکنان تیم میزبان خوش وبش ک

In [36]:
punctuation  = [ '+','=', '-' , '*' , '.' ,'?', '[', ']','(',')','{','}','<','>', '«' , '»' ,':' , "؟", "؛" , "،", "،" ]
signs = [ '!' , '@' , '#' , '$' , '%' , '^' , '&' , '*', '_', '\\' , '/' , '//' , '|' , "…" , "–" ,"_" , "   "]
numbers = ['۰' , '۱', '۲','۳', '۴', '۵','۶','۷','۸','۹']
signs_ir = ['٪' , ',' , '_','–' , '٫' , '"' ]
img_src = ['UFITNPF']
end = ['انتهای پیام']
# some_unknown_char = ['\u200c' , '\u200d' , '\u200e' , '\u200f']
english = list(string.ascii_lowercase) + list(string.ascii_uppercase)
not_used = punctuation + signs + numbers + signs_ir + img_src + english + end 
print(not_used)

['+', '=', '-', '*', '.', '?', '[', ']', '(', ')', '{', '}', '<', '>', '«', '»', ':', '؟', '؛', '،', '،', '!', '@', '#', '$', '%', '^', '&', '*', '_', '\\', '/', '//', '|', '…', '–', '_', '   ', '۰', '۱', '۲', '۳', '۴', '۵', '۶', '۷', '۸', '۹', '٪', ',', '_', '–', '٫', '"', 'UFITNPF', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'انتهای پیام']


In [37]:
stopwords = set(stopwords_list())
print(len(stopwords))

389


اینجا در ابتدا میبایستی ما یکسری از حروف‌های بلا استفاده رو حذف کنیم

In [38]:
# Make the change for the not_used word for us
for i in range(len(content)):
    for l in not_used:
        content[i] = content[i].replace(l,"")

In [ ]:
# make copy from the content that preprocess them (it used ^_^)
preprocess_content =copy.deepcopy(content)

In [ ]:
print(content[11])
print(content[9])

In [ ]:
# make the stem of the word
lstemmer = Stemmer()
lstemmer.stem('مظاهری')

In [31]:
# the verb stem
lemmatizer = Lemmatizer()
verbs = lemmatizer.lemmatize('مظاهری')
x = verbs.split('#')
print(x)

['مظاهر']


In [ ]:
# get tokenize by using the hazm word tokenize
normalizer = Normalizer()
dictionary_non_posting = collections.defaultdict(list)
for i in range(len(content)):
  tokenizes = []
  tokenizes = word_tokenize(content[i])
  tokenizes_count = dict(collections.Counter(tokenizes))
  for key , value in tokenizes_count.items():
    if key in dictionary_non_posting:
      value_old = dictionary_non_posting[key]
      dictionary_non_posting[key] = value + value_old
    else:
      dictionary_non_posting[key] = value 

In [ ]:
print((dictionary_non_posting['و']))
print(len(dictionary_non_posting))

130255
58734


In [ ]:
length_dict_non_posting = {k: v for k, v in sorted(dictionary_non_posting.items(), key=lambda item: item[1] , reverse=True)}
with open("mydict.txt", 'w') as f: 
    for key, value in length_dict_non_posting.items(): 
        f.write("%s:%s\n" % (key, value))

In [42]:
stop_words = ['آنها' , 'پیش' ,'پس' ,'هر' ,'او' ,'یا' ,'نیز' ,'وی' ,'ما' ,'خود' ,'هم' ,'تا' ,'آن' ,'بر' ,'برای' ,'را' ,'با' ,'که' ,'این' ,'از' ,'به' ,'در' ,'و' ]
stopwords.update(stop_words)
print(len(stopwords))

389


In [41]:
# to find position of each word in contents
def position_find(word_to_find , lists):
  # words = contents.split()
  return  [pos for pos, word in enumerate(lists, start=0) if ((word == word_to_find) )] 


In [ ]:
# get tokenize by using the hazm word tokenize (the postings list [docid , ferq , [position]])
normalizer = Normalizer()
dictionary_not_steam = collections.defaultdict(list)
for i in range(len(content)):
    tokenizes = []
    tokenizes = word_tokenize(content[i])
    tokenizes_count = dict(collections.Counter(tokenizes))
    for key , value in tokenizes_count.items():
      if((key not in stopwords)):
        index = []
        pos_index = position_find(key , tokenizes)
        index.append(i)
        index.append(value)
        index.append(pos_index)
        dictionary_not_steam[key].append(index)    

In [ ]:
print(dictionary_not_steam['و'])
print(len(dictionary_not_steam))
print(dictionary_not_steam['یحیی'])
print(len(dictionary_not_steam['یحیی']))

In [40]:
lstemmer = Stemmer()
lemmatizer = Lemmatizer()
def stem_stop(words_content):
  for j in range(len(words_content)):
        words_content[j] = lstemmer.stem(words_content[j])
  return words_content

In [43]:
# get tokenize by using the hazm word tokenize (the postings list [docid , ferq , [position]])
dictionary = collections.defaultdict(list)
for i in range(len(content)):
    tokenizes = []
    tokenizes = word_tokenize(content[i])
    tokenizes = stem_stop(tokenizes)
    tokenizes_count = dict(collections.Counter(tokenizes))
    for key , value in tokenizes_count.items():
        if key not in stopwords:
          index = []
          pos_index = position_find(key , tokenizes)
          index.append(i)
          index.append(value)
          index.append(pos_index)
          dictionary[key].append(index)

In [ ]:
print(len(dictionary))

In [44]:
# write the postings list in json 
from google.colab import drive
with open("posting-list.json", "w") as outfile:
    json.dump(dictionary, outfile)
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/IR/'

Mounted at /content/gdrive


In [ ]:
# Read postings list from json
with open('posting-list.json') as json_file:
    dictionary = json.load(json_file)

In [ ]:
print(len(dictionary[lstemmer.stem('و')]))
print((dictionary[lstemmer.stem('')]))
print(dictionary_not_steam[('بین‌الملل')])
print(len(dictionary_not_steam[('بین‌الملل')]))


In [ ]:
# the more priority
def sort_dict(dictionary):
   dicts = dictionary
   sorted_dicts = sorted(dicts, key = operator.itemgetter(0, 1) , reverse=False)
   all = 0
   for i in range(len(sorted_dicts)):
     all = all + sorted_dicts[i][1]
   return all , sorted_dicts

In [ ]:
# Enter With wrong keyboard language
def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

def convert_en_characters(input_str):
    mapping = {
        'q': 'ض',
        'w': 'ص',
        'e': 'ث',
        'r': 'ق',
        't': 'ف',
        'y': 'غ',
        'u': 'ع',
        'i': 'ه',
        'o': 'خ',
        'p': 'ح',
        '[': 'ج',
        ']': 'چ',
        'a': 'ش',
        's': 'س',
        'd': 'ی',
        'f': 'ب',
        'g': 'ل',
        'h': 'ا',
        'j': 'ت',
        'k': 'ن',
        'l': 'م',
        ';': 'ک',
        "'": 'گ',
        'z': 'ظ',
        'x': 'ط',
        'c': 'ز',
        'v': 'ر',
        'b': 'ذ',
        'n': 'د',
        'm': 'پ',
        ',': 'و',
        '?': '؟',
        'C': 'ژ',
        'H': 'آ',
    }
    return _multiple_replace(mapping, input_str)

In [ ]:
convert_en_characters("Cdlkhsjd;")

'ژیمناستیک'

In [ ]:
# one word query 
# this sorted query question by frequency
def query_one_word():
  query = input("enter a word for checking: ")
  query = convert_en_characters(query)
  time_start = datetime.datetime.now()
  normal_query = lstemmer.stem(query)
  print("Normal word to search {}".format(normal_query))
  # all ,sorted_dict = sort_dict(dictionary[normal_query])
  dicts = dictionary[normal_query]
  sorted_dict = sorted(dicts, key = operator.itemgetter(1, 2) , reverse=True)
  print(sorted_dict)
  time_finish = datetime.datetime.now()
  print("{} results in {} ms".format(len(sorted_dict), ((time_finish - time_start).total_seconds())*1000))
  print("id -> title\n")
  for i in sorted_dict:
    print("{} -> {}".format(i[0] , title[i[0]]))
  print(sorted_dict)



In [ ]:
query_one_word()

In [ ]:
# the dictionary values is the posting lists
# post_list = dictionary[key][i] 
# the i is iteated 
# return docID
def docID(post_list):
        return post_list[0]

In [ ]:
# the dictionary values is the posting lists
# post_list = dictionary[key][i] 
# the i is iteated 
# return list of position
def position(plist):
        return plist[2]    

In [ ]:
# position intersection algorithm
# get the to word position list and start find k position for p1 , p2
def position_intersect(p1,p2,k):
        answer = []                                                                     # answer <- ()
        len1 = len(p1)
        len2 = len(p2)
        i = j = 0 
        while i != len1 and j != len2:                                                  # while (p1 != nil and p2 != nil)
                if docID(p1[i]) == docID(p2[j]):
                        l = []                                                          # l <- ()
                        pp1 = position(p1[i])                                           # pp1 <- positions(p1)
                        pp2 = position(p2[j])                                           # pp2 <- positions(p2)
    
                        plen1 = len(pp1)
                        plen2 = len(pp2)
                        ii = jj = 0 
                        while ii != plen1:                                              # while (pp1 != nil)
                                while jj != plen2:                                      # while (pp2 != nil)
                                        if abs(pp1[ii] - pp2[jj]) <= k:                 # if (|pos(pp1) - pos(pp2)| <= k)
                                                l.append(pp2[jj])                       # l.add(pos(pp2))
                                        elif pp2[jj] > pp1[ii]:                         # else if (pos(pp2) > pos(pp1))
                                                break    
                                        jj+=1                                           # pp2 <- next(pp2)      
                                while l != [] and abs(l[0] - pp1[ii]) > k :             # while (l != () and |l(0) - pos(pp1)| > k)
                                        l.remove(l[0])                                  # delete(l[0])
                                for ps in l:                                            # for each ps in l
                                        answer.append([ docID(p1[i]), pp1[ii], ps ])    # add answer(docID(p1), pos(pp1), ps)
                                ii+=1                                                   # pp1 <- next(pp1)
                        i+=1                                                            # p1 <- next(p1)
                        j+=1                                                            # p2 <- next(p2)
                elif docID(p1[i]) < docID(p2[j]):                                       # else if (docID(p1) < docID(p2))
                        i+=1                                                            # p1 <- next(p1)                                                        
                else:
                        j+=1                                                            # p2 <- next(p2)
        return answer

In [ ]:
# multi word query 
# this sorted query question by frequency
def query_multi():
  sorted_dicts = []
  query = input("enter words for checking: ")
  time_start = datetime.datetime.now()
  query_split = query.split(" ")
  for q in range(len(query_split)):
    query_split[q] = convert_en_characters(query_split[q])
  k = 1
  if len(query_split) < 2:
    print("not enough word:(")
    return 
  for i in range(len(query_split)):
    normal_query = lstemmer.stem(query_split[i])
    print("Normal word to search {}".format(normal_query))
    all , diction =  sort_dict(dictionary[normal_query])
    print(all)
    print(diction)
    sorted_dicts.append((all , diction))
  print(sorted_dicts)
  sorted_dicts = sorted(sorted_dicts, key = lambda x: x[1])
  for x in range(len(sorted_dicts)-1):
    if(sorted_dicts[x][0] == 0):
      sorted_dicts.pop(x)
  print(len(sorted_dicts))
  print("sorted_dicts: ", sorted_dicts)
  if len(sorted_dicts) > 2:
    sorted_dicts1 = position_intersect(sorted_dicts[0][1] ,sorted_dicts[1][1] , k)
    sorted_dicts1 = sorted(sorted_dicts1, key = lambda x: x[1])
    sorted_dicts2 = position_intersect(sorted_dicts[1][1] ,sorted_dicts[2][1] , k)
    sorted_dicts2 = sorted(sorted_dicts2, key = lambda x: x[1])
    print("sorted_dicts1: ", sorted_dicts1)
    print("sorted_dicts2: ",sorted_dicts2)

  elif (len(sorted_dicts) == 2):
    sorted_dicts = position_intersect(sorted_dicts[0][1] ,sorted_dicts[1][1] , k)
    sorted_dicts = sorted(sorted_dicts, key = lambda x: x[1])
    print(sorted_dicts)
    res = collections.defaultdict(list)
    for x in range(len(sorted_dicts)):
      res[sorted_dicts[x][0]].append([sorted_dicts[x][1] , sorted_dicts[x][2]])
    res = sorted(res, key=lambda k: len(res[k]), reverse=True)
  time_finish = datetime.datetime.now()
  print("{} results in {} ms".format(len(res), ((time_finish - time_start).total_seconds())*1000))
  print("id -> title\n")
  for i in range(len(res)):
    print("{} -> {}".format(res[i] , title[res[i]]))


In [ ]:
query_multi()

In [ ]:
def zipfs_law():
  print(sorted(length_dict_non_posting, key=lambda x: x,reverse=True)[:20])


In [ ]:
zipfs_law()

In [ ]:
def heap_law():
  count = 1000
  normalizer = Normalizer()
  dictionary_heap = collections.defaultdict(list)
  for i in range(count):
    tokenizes = []
    tokenizes = word_tokenize(content[i])
    tokenizes = stem_stop(tokenizes)
    tokenizes_count = dict(collections.Counter(tokenizes))
    for key , value in tokenizes_count.items():
        if key not in stopwords:
          index = []
          pos_index = position_find(key , tokenizes)
          index.append(i)
          index.append(value)
          index.append(pos_index)
          dictionary_heap[key].append(index)
  sum = 0
  for key , value in dictionary_heap.items():
    for x in value:
      sum += x[1]
  
  print("sum for tokens for ",count," document is: ",sum," and length of vocabulary(M) is : ", len(list(dictionary_heap.keys())))

In [ ]:
heap_law()

## PHASE II

In [45]:
# Phase II
example = copy.deepcopy(dictionary[lstemmer.stem('یحیی')])
N = len(content)
for x in example:
    n = len(example)
    tfidf = (1+math.log((x[1]),10)) * math.log((N/n),10)
    x.append(tfidf)
print(dictionary[lstemmer.stem('یحیی')])
print(example)

[[1, 1, [126]], [47, 2, [160, 163]], [49, 1, [374]], [84, 2, [60, 63]], [128, 1, [50]], [132, 2, [55, 58]], [177, 2, [482, 485]], [223, 1, [6]], [226, 2, [19, 22]], [227, 1, [248]], [231, 1, [168]], [237, 1, [4]], [251, 2, [6, 9]], [268, 2, [4, 7]], [485, 2, [23, 100]], [532, 2, [246, 249]], [601, 1, [50]], [647, 2, [6, 9]], [652, 1, [15]], [653, 1, [183]], [654, 1, [163]], [656, 1, [89]], [659, 1, [104]], [665, 2, [28, 31]], [733, 1, [90]], [736, 1, [6]], [768, 1, [75]], [786, 2, [6, 9]], [791, 2, [83, 86]], [809, 2, [314, 317]], [814, 2, [43, 46]], [816, 1, [139]], [828, 1, [134]], [863, 2, [199, 202]], [864, 2, [78, 81]], [880, 1, [142]], [894, 2, [62, 240]], [928, 1, [14]], [932, 1, [57]], [964, 2, [337, 340]], [986, 1, [261]], [995, 2, [15, 18]], [1013, 2, [16, 83]], [1018, 1, [48]], [1043, 1, [361]], [1087, 1, [11]], [1105, 1, [6]], [1124, 3, [430, 815, 818]], [1133, 2, [16, 19]], [1157, 1, [30]], [1165, 1, [11]], [1177, 1, [35]], [1183, 1, [541]], [1191, 1, [137]], [1211, 1, [12

In [46]:
# Make the tfidf for all of the words
def calculate_tfidf(term_dict):
  N = len(content)
  doc_norm = [0] * N
  Champion_list = {}
  print("The contents size : " , N)
  for key , value in term_dict.items():
      value = term_dict[key]
      df = len(value)
      # print(term_dict[key] , key)
      if(df != 0):
        # the number of docs that we have value in 
        idf =  math.log((N/df),10)
        for x in value:
          # calcualte the tfidf for each docs
          tfidf = (1+math.log((x[1]),10)) * idf 
          doc_norm[x[0]] += tfidf**2 
          x.append(tfidf)
        value = sorted(value, key=operator.itemgetter(3), reverse=True)
        new_value = [idf, value]
        term_dict[key] = new_value
  return term_dict , doc_norm

In [47]:
# Make the documents size
def Doc_size(term_dict):
  N = len(content)
  doc_size = [0] * N
  for key,value in term_dict.items():
      for i in range(len(value)):
            doc_size[value[i][0]] = doc_size[value[i][0]] + value[i][1]
  return doc_size

In [48]:
# Nomilaze the vectors
def Vector_Normalize(term_dict , doc_norm):
    print("In Vector Normiliziton")
    print(term_dict['یح'])
    for key , value in term_dict.items():
      for i in range(len(value)):
        value[i][3] = value[i][3]/math.sqrt(doc_norm[value[i][0]])
    print(term_dict['یح'])
    return term_dict

In [49]:
# The Token Normalizetion
def Normalize_tokens(tokens):
  for i in range(len(tokens)):
    tokens[i] = lstemmer.stem(tokens[i])
  return tokens

In [50]:
# query vector Normalizition
def query_vector_normalize(query_dict):
  sigma = 0
  for key, value in query_dict.items():
    sigma = sigma + value**2
  for key, value in query_dict.items():
    query_dict[key] = value / math.sqrt(sigma)
  return query_dict

In [51]:
# Vector of the query 
def vectorize_query(query):
    query_token =  word_tokenize(query)
    normal_query_token = Normalize_tokens(query_token)
    # print(normal_query_token)
    term_freq_dict = dict(collections.Counter(normal_query_token))
    query_terms = {}
    for  term,freq in term_freq_dict.items():
        if term in dictionary.keys() and term not in stopwords:
            query_terms[term] = int(1+math.log(freq,10))
    # query_terms = query_vector_normalize(query_terms)
    return query_terms

In [52]:
# Calculate the Similiraty of query and docs
def CosineScore(query_dict , terms, doc_norm, K):
  N = len(content)
  scores_dict = {key:0 for key in range(N)}
  # scores_dict = [0 for i in range(0, N)]
  Final_score = []
  founded_doc = 0
  for key , value in query_dict.items():
    term = terms[key]
    idf_term = term[0]
    w_q = query_dict[key] * idf_term
    postings = term[1]
    # print(len(postings))
    champion_list = postings[:50]
    champion_list = postings
    for x in range(len(champion_list)):
      w_d =  champion_list[x][3]
      scores_dict[champion_list[x][0]] += w_q * w_d
      founded_doc += 1
  # print({k: v for k, v in sorted(scores_dict.items(), key=lambda item: item[1], reverse=True)})

  if founded_doc < K:
    print("Doc it`s not enough in Chmpion lists")
    for key , value in query_dict.items():
      if founded_doc < K:
        term = terms[key]
        idf_term = term[0]
        w_q = query_dict[key] * idf_term
        postings = term[1]
        above_doc = postings[:25]
        for x in range(len(above_doc)):
          w_d =  above_doc[x][3]
          scores_dict[above_doc[x][0]] += w_q * w_d
          founded_doc += 1
      
  for i in range(len(scores_dict)):
    if doc_norm[i] == 0 or scores_dict[i] == 0:
      continue
    scores_dict[i] = scores_dict[i] / math.sqrt(doc_norm[i])
    Final_score.append([i , scores_dict[i]])
  for i in range(len(Final_score)):
    for j in range(i+1, len(Final_score)):
      if Final_score[j][1] > Final_score[i][1]:
        Final_score[i], Final_score[j] = Final_score[j], Final_score[i]
  return Final_score

In [53]:
# Show the results 
def Show_res(Scores , K, normal_query):
  print("id -> title\n")
  print("The Five Top Related Docs\n")
  for i in range(K):
    print("{} -> {}".format(Scores[i][0] , title[Scores[i][0]]))
  find_secntences(Scores[0][0] , normal_query)
  print(title[Scores[0][0]])
  print(content[Scores[0][0]])

In [54]:
def find_secntences(doc_id , query_token):
  sentences = sent_tokenize(df.loc[doc_id].content)
  for sen in sentences:
    if(query_token[0] in sen):
        print(sen)

In [55]:
dictionarys = copy.deepcopy(dictionary)
dictionarys , doc_norm = calculate_tfidf(dictionarys)

The contents size :  7562


In [56]:
with open('tfidf.pickle', 'wb') as handle:
    pickle.dump(dictionarys, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('doc_norm.pickle', 'wb') as handle:
    pickle.dump(doc_norm, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
query = input("enter a word for checking: ")
time_start = datetime.datetime.now()
normal_query = lstemmer.stem(query)
print("Normal word to search {}".format(normal_query))
query_dict = vectorize_query(query)
K = 5
Scores = CosineScore(query_dict , dictionarys , doc_norm, K)
time_finish = datetime.datetime.now()
print("{} results in {} ms".format(len(Scores), ((time_finish - time_start).total_seconds())*1000))
Show_res(Scores, K, normal_query)

In [ ]:
query_one_word()  

In [ ]:
query_multi()

##This is the Preferential part of Phase II

####my model

In [ ]:
pip install --upgrade gensim

In [1]:
import pickle
import multiprocessing
from gensim.models import Word2Vec
import time
from numpy.linalg import norm

In [8]:
stopwords = set(stopwords_list())
stop_words = ['آنها' , 'پیش' ,'پس' ,'هر' ,'او' ,'یا' ,'نیز' ,'وی' ,'ما' ,'خود' ,'هم' ,'تا' ,'آن' ,'بر' ,'برای' ,'را' ,'با' ,'که' ,'این' ,'از' ,'به' ,'در' ,'و' ]
stopwords.update(stop_words)

In [14]:
traning_data = []
for doc_id in range(len(content)):
  word_list = []
  word_list = word_tokenize(content[doc_id])
  word_list = stem_stop(word_list)
  for word in word_list:
    if word in stopwords:
      word_list.remove(word)
  traning_data.append(word_list)

In [15]:
print(len(traning_data))

7562


In [18]:
with open('my_traning_data.pickle', 'wb') as handle:
    pickle.dump(traning_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [110]:
handle = open('my_traning_data.pickle', 'rb')
traning_data = pickle.load(handle)

In [19]:
cores = multiprocessing.cpu_count()
print("# cores " , cores)

# cores  2


In [20]:
docs_num = len(traning_data)
tokens_num = sum([len(x) for x in traning_data])
print(docs_num)
print(tokens_num)

7562
1755713


In [111]:
%%time
w2v_model = Word2Vec(min_count = 1,
                     window = 5,
                     vector_size = 300,
                     alpha = 0.03,
                     workers = 2)
w2v_model.build_vocab(traning_data)
w2v_model_vocab_size = len(w2v_model.wv)
print(w2v_model_vocab_size)

40373
CPU times: user 1.25 s, sys: 4.97 ms, total: 1.26 s
Wall time: 1.27 s


In [112]:
start = time.time()
w2v_model.train(traning_data , total_examples=w2v_model.corpus_count, epochs= 20)
end = time.time()
print("{} s".format(end-start))

70.4228286743164 s


In [113]:
w2v_model.save('w2v_myModel.model')
# w2v_model = Word2Vec.load('w2v_myModel.model')

In [114]:
# the word vector lookup table
len(w2v_model.wv)

40373

In [115]:
(w2v_model.wv['یح'])

array([-3.45066428e-01,  7.43347704e-02, -5.85016966e-01,  3.98130327e-01,
        1.30105770e+00, -5.23056805e-01,  2.07763072e-02, -1.12160794e-01,
        2.87920266e-01, -6.32298112e-01,  1.21837461e+00,  1.38908780e+00,
       -4.75313753e-01,  2.68529773e-01,  1.00922894e+00, -4.61911410e-01,
       -4.11071986e-01,  6.89613700e-01, -7.72028923e-01,  5.76258242e-01,
        9.31119084e-01, -1.30945474e-01, -8.72690201e-01, -1.57994461e+00,
        5.64066410e-01, -3.06162611e-02, -5.25149345e-01,  2.37967387e-01,
       -1.69981495e-01, -8.75502110e-01, -1.48857310e-01, -3.71986777e-01,
        6.03224993e-01,  3.45195591e-01,  7.86476552e-01,  5.46946108e-01,
       -5.84725678e-01, -1.11062264e+00, -4.76734459e-01,  1.53512910e-01,
       -2.24046573e-01, -7.89780021e-01, -1.50433302e-01, -1.28473186e+00,
        3.45782071e-01, -5.46696424e-01, -5.58586180e-01, -5.41899949e-02,
       -6.95443630e-01, -2.35190347e-01, -9.91130769e-01,  1.16512917e-01,
        2.95556575e-01,  

##Check the similarity

In [116]:
# the same word similar word to this
w2v_model.wv.most_similar('بورس')

[('بهادار', 0.6305003762245178),
 ('معامل', 0.46025604009628296),
 ('معاملات', 0.4302796721458435),
 ('اوراق', 0.42747336626052856),
 ('فرابورس', 0.41988250613212585),
 ('بازار', 0.4010249972343445),
 ('فنارو', 0.38652268052101135),
 ('\u200cعال', 0.371860146522522),
 ('کارگزار', 0.3655050992965698),
 ('فرو', 0.3618781566619873)]

In [117]:
score = w2v_model.wv.similarity('پرسپولیس' ,'استقلال')
(score+1)/2

0.7355090826749802

###Doc_embedding

In [118]:
handle = open('tfidf.pickle' , 'rb')
docs_tf_idf = pickle.load(handle)

In [119]:
docs_tf_idf['یح'][0]

1.9243941635871917

In [71]:
docs_tfidf = []
for i in range(len(content)):
  dict_word = {}
  for key, value in docs_tf_idf.items():
    word = key
    for x in value[1]:
      if x[0] == i:
        dict_word[word] = x[3]
  docs_tfidf.append(dict_word)


In [72]:
with open('new_tfidf_doc.pickle', 'wb') as handle:
    pickle.dump(docs_tfidf, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [120]:
print(docs_tfidf[0])

{'مسل': 1.6911159521900536, 'مجدم': 4.480904391835638, 'گفت\u200cوگو': 0.9569501975429147, 'خبرنگار': 0.6299279374255988, 'ورزش': 0.9210293859664216, 'خبرگزار': 0.33755690524988785, 'فارس': 0.31043494895952184, 'شکس': 1.359384181217871, 'صفر': 1.1519094639999445, 'نف': 1.8452598628640926, 'مسجدسل': 3.2777322171476033, 'مقابل': 1.2037476377190055, 'سپاه': 2.377490339353227, 'اظهار': 0.8141632890985091, 'دا': 0.7043478893589665, 'نمی\u200cکرد': 2.6376800124829596, 'اتفاق': 1.7423738342862285, 'بیفتد': 1.8533308077617463, 'زود': 1.295437899057894, 'گل': 1.049976776491197, 'خورد': 1.3789495904081128, 'غافلگیر': 3.124443594266751, 'پ': 0.807846720836547, 'مسابق': 0.9411187810091702, 'تن': 0.8356675996333366, 'هفته': 1.192158231137147, 'تمرین': 2.152447805259988, 'شرایط': 1.2637834164717143, 'نمی\u200cتو': 1.720274180931267, 'ت': 1.0996508381954209, 'بقیه': 1.7817266600184603, 'تیم': 1.3661554470692752, 'مقایسه': 1.7646933207196798, 'برخ': 0.7880259651981101, 'حت': 1.4764429510772679, 'ب': 0

In [121]:
docs_embbedding = []
for doc in docs_tfidf:
  doc_vec = np.zeros(300)
  weights_sum = 0
  for token,weight in doc.items():
    try:
      doc_vec += w2v_model.wv[token] * weight
      weights_sum += weight
    except Exception:
      pass
  docs_embbedding.append(doc_vec/weights_sum)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


###Docs_similarity

In [122]:
print(docs_embbedding[0])

[ 0.18131304  0.05900072  0.10226667 -0.17016551  0.2744504   0.15033492
 -0.16081591 -0.1103604  -0.19343617 -0.177293    0.2461186  -0.0973968
  0.37071955  0.46319689 -0.27752888 -0.21101814 -0.20920031  0.57790799
 -0.18398261 -0.2341126  -0.11455104 -0.31511761 -0.4721922   0.10455449
 -0.6479559   0.0835621  -0.1072139  -0.30740328 -0.15245467  0.0936793
 -0.25603821  0.09869928  0.32098623 -0.07939365 -0.21867852  0.30041877
 -0.55895036 -0.13572233 -0.6898137   0.11049916  0.00832321  0.1578386
 -0.12607572 -0.108381    0.03734651  0.05265441 -0.34559598 -0.20046456
 -0.22665919 -0.12113002 -0.38120905 -0.43400223  0.08972791  0.27563404
  0.04629244  0.01057582 -0.46336841 -0.17100977 -0.05262841 -0.17250211
 -0.05545516 -0.0582168  -0.08360641 -0.00418272 -0.11899657  0.03086686
  0.23155678 -0.21816774  0.12817395  0.21450311 -0.03996406  0.13136348
  0.31880362  0.31410266 -0.14721934  0.15275533  0.29252495  0.12353271
  0.25230951  0.14486119  0.17767684 -0.10232526  0.34

In [123]:
def similarity(doc1, doc2):
  similarity_score = np.dot(doc1,doc2)/(norm(doc1) * norm(doc2))
  return (similarity_score+1)/2

In [124]:
similarity(docs_embbedding[12] ,docs_embbedding[10])

0.9275508112729814

In [127]:
# Show the results 
def Show_res_point(Scores , K):
  print("id -> title\n")
  print("The Five Top Related Docs\n")
  for i in range(K):
    print("{} -> {}".format(Scores[i][0] , title[Scores[i][0]]))

In [142]:
query = input("enter a word for checking: ")
time_start = datetime.datetime.now()
normal_query = lstemmer.stem(query)
print("Normal word to search {}".format(normal_query))
query_dict = vectorize_query(query)
print(query_dict)
query_vec = np.zeros(300)
weights_sum = 0
for token,weight in query_dict.items():
  try:
    query_vec += w2v_model.wv[token] * weight
    weights_sum += weight
  except Exception:
    pass
Scores = []
for i in range(len(docs_embbedding)):
  Scores.append((i ,similarity( docs_embbedding[i],query_vec)))
print(len(Scores))
time_finish = datetime.datetime.now()
print("{} results in {} ms".format(len(Scores), ((time_finish - time_start).total_seconds())*1000))
Scores = sorted(Scores,key = lambda x: x[1], reverse=True)
Show_res_point(Scores , K=5)

enter a word for checking: آیا پرسپولیس از این مرحله لیگ قهرمانان صعود می‌کند ؟
Normal word to search آیا پرسپولیس از این مرحله لیگ قهرمانان صعود می‌کند ؟
{'پرسپولیس': 1, 'مرحله': 1, 'لیگ': 1, 'قهرمان': 1, 'صعود': 1}
7562
7562 results in 101.342 ms
id -> title

The Five Top Related Docs

661 -> النصر مدافعش را برای نیمه نهایی لیگ قهرمانان آسیا از دست داد
742 -> ژاردیم: فردا شاهد رقابت بهترین تیم‌های آسیا خواهیم بود/به دنبال پیروزی مقابل پرسپولیس هستیم
766 -> الهلال با آمار موفقیت آمیز 71 درصدی مقابل پرسپولیس+عکس
649 -> واکنش جالب AFC به برد قاطع الهلال مقابل پرسپولیس+عکس
669 -> حضور هواداران الهلال  در ورزشگاه الملز مقابل پرسپولیس+فیلم


####your model

In [99]:
!unzip "/content/gdrive/MyDrive/word2vec/word2vec_model_hazm.zip" -d "/content/gdrive/MyDrive/word2vec/word2vec"

Archive:  /content/gdrive/MyDrive/word2vec/word2vec_model_hazm.zip
  inflating: /content/gdrive/MyDrive/word2vec/word2vec/w2v_150k_hazm_300_v2.model  
  inflating: /content/gdrive/MyDrive/word2vec/word2vec/w2v_150k_hazm_300_v2.model.trainables.syn1neg.npy  
  inflating: /content/gdrive/MyDrive/word2vec/word2vec/w2v_150k_hazm_300_v2.model.wv.vectors.npy  


In [129]:
w2v_model_150K = Word2Vec.load('/content/gdrive/MyDrive/word2vec/word2vec/w2v_150k_hazm_300_v2.model')

In [130]:
docs_embbedding_150K = []
for doc in docs_tfidf:
  doc_vec = np.zeros(300)
  weights_sum = 0
  for token,weight in doc.items():
    try:
      doc_vec += w2v_model_150K.wv[token] * weight
      weights_sum += weight
    except Exception:
      pass
  docs_embbedding_150K.append(doc_vec/weights_sum)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [143]:
query = input("enter a word for checking: ")
time_start = datetime.datetime.now()
normal_query = lstemmer.stem(query)
print("Normal word to search {}".format(normal_query))
query_dict = vectorize_query(query)
print(query_dict)
query_vec = np.zeros(300)
weights_sum = 0
for token,weight in query_dict.items():
  try:
    query_vec += w2v_model_150K.wv[token] * weight
    weights_sum += weight
  except Exception:
    pass
Scores = []
for i in range(len(docs_embbedding_150K)):
  Scores.append((i ,similarity( docs_embbedding_150K[i],query_vec)))
print(len(Scores))
time_finish = datetime.datetime.now()
print("{} results in {} ms".format(len(Scores), ((time_finish - time_start).total_seconds())*1000))
Scores = sorted(Scores,key = lambda x: x[1], reverse=True)
Show_res_point(Scores , K=5)

enter a word for checking: آیا پرسپولیس از این مرحله لیگ قهرمانان صعود می‌کند ؟
Normal word to search آیا پرسپولیس از این مرحله لیگ قهرمانان صعود می‌کند ؟
{'پرسپولیس': 1, 'مرحله': 1, 'لیگ': 1, 'قهرمان': 1, 'صعود': 1}
7562
7562 results in 94.409 ms
id -> title

The Five Top Related Docs

3595 -> پیروزی یاران موسوی با درخشش لژیونر ایران 
650 -> بهترین بازیکن دیدار پرسپولیس-الهلال مشخص شد
3827 -> برزیل مقتدرانه قهرمان لیگ ملت های والیبال شد/ لهستان زانو زد
3564 -> خداحافظی تلخ آلکنو با کازان پس از پنجمی زنیت در سوپر لیگ روسیه
742 -> ژاردیم: فردا شاهد رقابت بهترین تیم‌های آسیا خواهیم بود/به دنبال پیروزی مقابل پرسپولیس هستیم
